# Creating an agent
This notebook will go through the how to create a new agent within the tomsup framework. In this tutorial we will be making an reversed win-stay, lose-switch agent, e.g. an win-switch, lose-stay agent.

This guides assumes a basic understanding of classes in python, if you don't know these or need to recap we suggest examing this [chapter](http://hplgit.github.io/primer.html/doc/pub/class/._class-readable002.html) in the free ebook a byte of python

Let us first import the package:

In [1]:
# assuming you are in the github folder change the path - not relevant if tomsup is installed via. pip
import os

os.chdir("..")  # go back one folder

import tomsup as ts

Now lets first take a look at the current win-stay, lose-switch (WSLS) agent:

In [2]:
sigmund = ts.WSLS()  # create agent

# inspect sigmund
print(f"sigmund is an class of type: {type(sigmund)}")  # f is for format
if isinstance(sigmund, ts.Agent):
    print(f"but sigmund is also of has the parent class ts.Agent")

sigmund is an class of type: <class 'tomsup.agent.WSLS'>
but sigmund is also of has the parent class ts.Agent


As we can see sigmund is a WSLS agent with the parent class tsAgent. This us some benefits as WSLS inherit some of the attributes of the parent class, such as the ability to save play history and the ability to reset the agents. To see more of the inherited methods see help(ts.WSLS).

## Creating a new class
Now let's try to create our own agent one bit at a time (if you are confortable with classes simply jump to 'The final reversed WSLS):

In [3]:
import numpy as np


class ReversedWSLS(ts.Agent):  # make sure that the parent class is ts.Agent
    """
    ReversedWSLS: Win-switch, lose-stay.

    This agent is a reversed win-stay, lose-switch agent, which ...
    """

    # add a docstring which explains the agent
    pass  # we will later replace this pass with something else


freud = ReversedWSLS()
print(f"is freud an Agent? {isinstance(freud, ts.Agent)}")

is freud an Agent? True


### Add initialization
Let's add an initalization of the agent. These are things which should be created prior to the agent competing.

In [4]:
class ReversedWSLS(ts.Agent):
    """
    ReversedWSLS: Win-switch, lose-stay.

    This agent is a reversed win-stay, lose-switch agent, which ...
    """

    def __init__(self, first_move, **kwargs):  # initalize the agent
        self.strategy = "ReversedWSLS"  # set the strategy name

        # set internal parameters
        self.first_move = first_move

        super().__init__(
            **kwargs
        )  # pass additional argument the ts.Agent class (could e.g. include 'save_history = True')
        self._start_params = {
            "first_move": first_move,
            **kwargs,
        }  # save any starting parameters used when the agent is reset


freud = ReversedWSLS(first_move=1)
print(f"what is freud's first move? {freud.first_move}")
print(f"what is freud's starting parameters? {freud.get_start_params()}")
print(f"what is freud's strategy? {freud.get_strategy()}")

what is freud's first move? 1
what is freud's an starting parameters? {'first_move': 1}
what is freud's strategy? ReversedWSLS


In the above you sucessfully created an freud as an agent and that his first move is 1. We also see that functions such as the ```get_start_params()``` from the ts.Agent is inherited by the new agent. 


**Note** that we have set ```**kwargs```, this simply means that function accept additional arguments, e.g. ```save_history = True```.
These arguments are then passed to the ```super()__init__()```, which initialize the parent class (in this case the ts.Agent class) as well as the ```_start_params``` which is the starting parameters. The starting parameter are used when resetting the agent, which is relevant e.g. when setting up a tournament settings. 

#### Add a compete function
All agent naturally need a compete function. Let us add one to the agent

In [5]:
class ReversedWSLS(ts.Agent):
    """
    ReversedWSLS: Win-switch, lose-stay.

    This agent is a reversed win-stay, lose-switch agent, which ...
    """

    def __init__(self, first_move, **kwargs):  # initalize the agent
        self.strategy = "ReversedWSLS"  # set the strategy name

        # set internal parameters
        self.first_move = first_move

        super().__init__(
            **kwargs
        )  # pass additional argument the ts.Agent class (could e.g. include 'save_history = True')
        self._start_params = {
            "first_move": first_move,
            **kwargs,
        }  # save any starting parameters used when the agent is reset

    def compete(self, p_matrix, op_choice=None, agent=0):
        """
        win-switch, lose-stay strategy, with the first move being set when the class is initilized (__init__())

        p_matrix is a PayoffMatrix
        op_choice is either 1 or 0
        agent is either 0 or 1 and indicated the perpective of the agent in the game (whether it is player 1 og 2)
        """
        if (
            self.choice is None
        ):  # if a choice haven't been made: Choose the redifined first move
            self.choice = self.first_move  # fetch from self
        else:  # if a choice have been made:
            payoff = p_matrix.payoff(
                self.choice, op_choice, agent
            )  # calculate payoff of last round
            if payoff == 1:  # if the agent won then switch
                self.choice = (
                    1 - self.choice
                )  # save the choice in self (for next round)
                # also save any other internal states which you might
                # want the agent to keep for next round in self
        self._add_to_history(
            choice=self.choice
        )  # save action and (if any) internal states in history
        # note that _add_to_history() is not intented for
        # later use within the agent
        return self.choice  # return choice which is either 1 or 0


freud = ReversedWSLS(first_move=1)  # create the agent

# fetch payoff matrix for the pennygame
penny = ts.PayoffMatrix(name="penny_competitive")
print(
    "This is the payoffmatrix for the game (seen from freud's perspective):",
    penny()[0, :, :],
    sep="\n",
)

# have freud compete
choice = freud.compete(penny)
print(f"what is freud's choice the first round? {choice}")
choice = freud.compete(penny, op_choice=1)
print(f"what is freud's choice the second round if his opponent chose 1? {choice}")

This is the payoffmatrix for the game (seen from freud's perspective):
[[-1  1]
 [ 1 -1]]
what is freud's choice the first round? 1
what is freud's choice the second round if his opponent chose 1? 1


In the above script we add freud's compete function, which for the first round choses his own move and for future moves it uses the win-switch, lose-stay strategy. It then return either a 0 or 1 depending on whether is choses e.g. right or left hand in the penny game. It is important that the agent does only return 0 or 1 in its compete function otherwise the agent will not function in the context of the package. 

**Note** the ```self._add_to_history(choice = self.choice)```, which indicated which variables I would like to add to the agent history, assuming save history is set to ```True```. In this case we would like to.

Finally when you have the ```__init__()``` and the ```compete()``` working you can add any additional function you might want your agent to have, for example you will se that we have added the ```get_first_move()```, which is a helper function to extract the first move of the agent.

## The final reversed WSLS
The following is the finalized version of the win-switch, lose-stay agent.

In [6]:
import numpy as np


class ReversedWSLS(ts.Agent):
    """
    ReversedWSLS: Win-switch, lose-stay.

    This agent is a reversed win-stay, lose-switch agent, which ...

    Examples:
    >>> waade = ReversedWSLS(first_move = 1)
    >>> waade.compete(op_choice = None, p_matrix = penny)
    1
    """

    def __init__(self, first_move, **kwargs):
        self.strategy = "ReversedWSLS"

        # set internal parameters
        self.first_move = first_move

        super().__init__(
            **kwargs
        )  # pass additional argument the ts.Agent class (could e.g. include 'save_history = True')
        self._start_params = {
            "first_move": first_move,
            **kwargs,
        }  # save any starting parameters used when the agent is reset

    def compete(self, p_matrix, op_choice=None):
        if (
            self.choice is None
        ):  # if a choice haven't been made: Choose the redifined first move
            self.choice = self.first_move  # fetch from self
        else:  # if a choice have been made:
            payoff = p_matrix.payoff(
                self.choice, op_choice, 0
            )  # calculate payoff of last round
            if payoff == 1:  # if the agent won then switch
                self.choice = (
                    1 - self.choice
                )  # save the choice in self (for next round)
                # also save any other internal states which you might
                # want the agent to keep for next round in self
        self._add_to_history(
            choice=self.choice
        )  # save action and (if any) internal states in history
        # note that _add_to_history() is not intented for
        # later use within the agent
        return self.choice  # return choice

    # define any additional function you wish the class should have
    def get_first_move(self):
        return self.first_move

## Test your knowlegde

1) Create an agent called Random, which simply choose randomly

2) Check that it is an agent and that the compete function work

3) Have the agent compete against another agent within the package using the ```ts.compete()```, which one win?

# FAQ

- I have developed an agent which I would like to include in your package
Sounds lovely, we would love to include the agent. Feel free to make a pull request on Github or contact us at kennethcenevoldsen@gmail.com.